In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys

import torch
from torch import nn

## Access src directory from ./notebooks/ folder
from pathlib import Path
sys.path.append(str(Path('.').absolute().parent))

from src.SnipByC import SNIP
from src.models.LeNet import LeNet300100, LeNet5Caffe
from src.optimization import MeanEvaluator

import numpy as np
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST, CIFAR10
import torchvision.transforms as transforms

In [ ]:
dataset = MNIST

data_transforms = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = dataset('../data/', train=True, download=True, transform=data_transforms)
train_data_loader = torch.utils.data.DataLoader(train_dataset,
                                                batch_size=128, shuffle=True, num_workers=2)

In [ ]:
model = LeNet300100()
#list(model.parameters())
snip = SNIP(model)
snip.compute_mask(train_data_loader, 10)